In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter
from sklearn.model_selection import train_test_split
import nltk
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
from gensim import models

In [2]:
NUM_WORKERS = 8
TEST_SIZE = 0.3
LABEL_COLUMN_NAME = "oh_label"
TEXT_COLUMN_NAME = "Text"
DATASET_PATH = "../Data/ver1.csv"

df= pd.read_csv(DATASET_PATH, index_col=False)
df = df.dropna()
df.drop(["Unnamed: 0", "index"],axis=1, inplace = True)

sience we dont have a good pre-trained doc2vec model we will use the pre-trained doc2vec for each word
and take the mean of the output vector

['Im',
 '12',
 'and',
 'i',
 'can',
 'understand',
 'it',
 'perfectly',
 '.',
 'You',
 'should',
 'learn',
 'english',
 'instead',
 'of',
 'making',
 'us',
 'dumb',
 'it',
 'down',
 'for',
 'you',
 '.',
 '#',
 'mkr',
 'Boy',
 ',',
 '@',
 'FourinHandyou',
 'sure',
 'know',
 'how',
 'to',
 'dish',
 'out',
 'an',
 'insult',
 '``',
 'your',
 'sausage',
 'was',
 'a',
 'little',
 'short',
 'of',
 'fat',
 '...',
 "''",
 'Glad',
 'they',
 'were',
 "n't",
 'male',
 'contestants',
 '!',
 'Fuck',
 'you',
 'all',
 '.',
 'This',
 'site',
 'is',
 'full',
 'of',
 'stuck',
 'up',
 'cunts',
 ',',
 'who',
 'for',
 'some',
 'unknown',
 'reason',
 'think',
 'being',
 'a',
 'part',
 'of',
 'wikipedia',
 'promotes',
 'them',
 'to',
 'President',
 'of',
 'the',
 'cyberspace',
 '.',
 'Sad',
 'pricks',
 ',',
 'go',
 'fuck',
 'yourselves',
 '.',
 'I',
 'wo',
 "n't",
 'be',
 'returning',
 'to',
 'this',
 'site',
 '.',
 'So',
 'off',
 'you',
 'all',
 'go',
 'to',
 'self',
 'congratulate',
 'yourselves',
 '.',
 '`

In [9]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [7]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged


In [10]:
df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].swifter.apply(lambda x: get_average_word2vec(x, word2vec,generate_missing=True))

Pandas Apply: 100%|██████████| 100000/100000 [06:56<00:00, 239.81it/s]


0        [0.015312941419345982, 0.22473520653347204, 0....
1        [0.015332999627281975, 0.21280927531473268, 0....
2        [-0.007219608801433893, 0.19396265298012924, 0...
3        [0.017972469479406367, 0.22978346115482676, 0....
4        [0.03493955455862177, 0.20538398582791106, 0.1...
                               ...                        
99995    [0.03001815631774615, 0.20729014185692446, 0.1...
99996    [0.0435618249695891, 0.2327276160123481, 0.181...
99997    [0.03214820638988943, 0.24006346296471937, 0.1...
99998    [-0.07424437940884678, 0.12839322934619346, 0....
99999    [0.006174476764086151, 0.22336781633304606, 0....
Name: Text, Length: 100000, dtype: object

In [15]:
X,y = df[TEXT_COLUMN_NAME],df[LABEL_COLUMN_NAME]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,test_size=0.5)

In [16]:
x_neg_weight = y[y==1].shape
x_pos_weight = y[y==0].shape
x_neg_weight[0]/x_pos_weight[0]
weights = {0: (x_neg_weight[0]/(x_pos_weight[0]+x_neg_weight[0])),1: (x_pos_weight[0]/(x_pos_weight[0]+x_neg_weight[0]))}
print(weights)

{0: 0.26771, 1: 0.73229}


In [21]:
x_train = np.expand_dims(np.stack(x_train),axis=2)
x_test = np.expand_dims(np.stack(x_test),axis=2)

In [ ]:
x_train.squeeze().shape

In [23]:
# strategy = tf.distribute.MirroredStrategy()
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(200, activation="relu",input_shape=(300,1)))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300, 200)          400       
_________________________________________________________________
dense_1 (Dense)              (None, 300, 100)          20100     
_________________________________________________________________
dropout (Dropout)            (None, 300, 100)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 300, 50)           5050      
_________________________________________________________________
dense_3 (Dense)              (None, 300, 10)           510       
_________________________________________________________________
dense_4 (Dense)              (None, 300, 1)            11        
Total params: 26,071
Trainable params: 26,071
Non-trainable params: 0
____________________________________________________

In [25]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
history = model.fit(x_train,y_train,batch_size=200,epochs=500,
                    validation_split=0.05,class_weight=weights,
                    use_multiprocessing=True,workers=8,callbacks=[callback])

Epoch 1/500


ValueError: in user code:

    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\sit